In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_community.llms import HuggingFaceHub
from crewai import LLM
import os

# Setting up the LLMs to use for our agents.
# We can also keep different LLMs for different agents

# llm = HuggingFaceHub(
#     repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
#     huggingfacehub_api_token = os.getenv("HF_TOKEN"),
#     task = "text-generation",
# )

llm = LLM(model = "openai/gpt-4o-mini")

### Initialize the Agents

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
# Setting up the Content Planner agent which will plan on writing for the article
# The agent's goal is to plan by collecting information for the Writer

planner = Agent(
    role = "Content Planner",
    goal = "Plan engaging and factually accurate content on {topic}",
    backstory = """You're working on planning a blog article about the topic: {topic}. 
    You collect information that helps the audience learn something and make informed decisions. 
    Your work is the basis for the Content Writer to write an article on this topic.""",
    llm = llm,
    allow_delegation = False,
	verbose = True
)

In [ ]:
# Setting up the Content Writer Agent
# It is tasked to use the information given by the content planner and write an article about the topic

writer = Agent(
    role = "Content Writer",
    goal = """Write insightful and factually accurate opinion piece about the topic: {topic}""",
    backstory = """You're working on a writing a new opinion piece about the topic: {topic}. 
    You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic. 
    You follow the main objectives and direction of the outline, as provide by the Content Planner. 
    You also provide objective and impartial insights and back them up with information provide by the Content Planner. 
    You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.""",
    llm = llm,
    allow_delegation = False,
    verbose = True
)

In [ ]:
# Setting up the Editor Agent
# It is tasked to review the article written by the Content Writer Agent and edit it if necessary
editor = Agent(
    role = "Editor",
    goal = "Edit a given blog post to align with the writing style of the organization.",
    backstory = """You are an editor who receives a blog post from the Content Writer. 
    Your goal is to review the blog post to ensure that it follows journalistic best practices, 
    provides balanced viewpoints when providing opinions or assertions, and also avoids major controversial topics 
    or opinions when possible.""",
    llm = llm,
    allow_delegation = False,
    verbose = True
)

### Initialize the Tasks

In [ ]:

plan = Task(
    description = """1. Prioritize the latest trends, key players, and noteworthy news on {topic}. 
    2. Identify the target audience, considering their interests and pain points. 
    3. Develop a detailed content outline including an introduction, key points, and a call to action. 
    4. Include SEO keywords and relevant data or sources.""",
    expected_output = """A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.""",
    agent = planner,
)

In [ ]:
write = Task(
    description = """1. Use the content plan to craft a compelling blog post on {topic}. 
    2. Incorporate SEO keywords naturally. 
    3. Sections/Subtitles are properly named in an engaging manner. 
    4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion. 
    5. Proofread for grammatical errors and alignment with the brand's voice.""",
    expected_output = "A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent = writer,
)

In [ ]:
edit = Task(
    description = "Proofread the given blog post for grammatical errors and alignment with the brand's voice.",
    expected_output = "A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent = editor
)

### Create the Crew

In [ ]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = True
)

In [ ]:
result = crew.kickoff(inputs = {"topic": "Artificial Intelligence"})

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)